# Installation

In [ ]:
pip install google-api-python-client

In [1]:
pip install oauthlib

Note: you may need to restart the kernel to use updated packages.


# OAuth 2.0 setup

In [48]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

In [49]:
scopes = ['https://www.googleapis.com/auth/calendar']

In [50]:
# This flow is used to obtain user consent and generate credentials that allow the application to access certain APIs on behalf of the user.

flow = InstalledAppFlow.from_client_secrets_file("client_secret.json",scopes = scopes)

In [51]:
# Saving credentials into a variable so as to avoid manual running
# credentials = flow.run_local_server(port=0) 

In [52]:
# Saving the credentials into a pickle file

import pickle

# pickle.dump(credentials,open("credential_token.pkl","wb"))
credentials = pickle.load(open("credential_token.pkl","rb"))

In [53]:
# Generating a service
service = build("calendar","v3",credentials = credentials)


# Get my Calendar

In [54]:
calendar_response = service.calendarList().list().execute()
# calendar_response

# Get all My calendar events

In [55]:
calendar_id = calendar_response['items'][0]['id']


In [56]:
# Google Calendar API to retrieve a list of events from a specific calendar based on a search query.
calendar_event_response = service.events().list(calendarId = calendar_id,q="birthday",orderBy="startTime",singleEvents="true").execute()

In [57]:
birthday_events = calendar_event_response['items']

In [2]:
import emoji

In [3]:
# Customised birthday wish
def get_birthday_msg(name):
    
    return f"Happy birthday, {name}!🥳🎂 May your special day be filled with joy, love, and wonderful memories😍❤. Wishing you happiness and success in the year ahead😇. Enjoy your celebration!🎊🎉"


In [4]:
import pywhatkit

In [7]:
def send_message_to_whatsapp(name,mobile_number):
    
    message = get_birthday_msg(name)

    hours = 12
    minutes = 0

    # Send msg at specific time (11:59 PM)
#     pywhatkit.sendwhatmsg(mobile_number,message,hours,minutes)

    # Send msg instantly
    pywhatkit.sendwhatmsg_instantly(mobile_number,message)

send_message_to_whatsapp("Alice","+919550731837")

In [62]:
from datetime import datetime

# Converting the datetime from str to <datetime> type
def extract_date(date_time_string):
    
    # Parse the string and extract the date
    datetime_obj = datetime.strptime(date_time_string,"%Y-%m-%dT%H:%M:%SZ")
    
    date_only = datetime_obj.date()

    # Return the extracted date
    return date_only
    
    

# Schedule the birthday wishes

In [63]:
# Schdeule the birthday wish to run at a specific date and time


In [68]:
def extract_details():
    
    # Extracting mobile number
    mobile_number = person_event['description']
    mobile_number = mobile_number.split(' ')[-1]

    # Extracting name of the birthday boy/girl
    name = person_event['summary']
    name = name.split("'")[0]

    # Extract time
    birthdate = person_event['start']['dateTime']
    birthdate = extract_date(birthdate)
    
    return mobile_number,name,birthdate

In [ ]:
import schedule
import time
from datetime import datetime, timedelta,date

    
# Retrieve the name and contact number for automating birthday wishes
for person_event in birthday_events:

    number,name,birthdate = extract_details()
    
    # Calculate the scheduled datetime for wishes (11:59 PM on the previous day)
    scheduled_date = birthdate - timedelta(days=1)
    
    # Get the current time
    current_date = date.today()
    
    if scheduled_date == current_date:
        
        # Schedule the wishes to be sent at the current time, RUN EVERYDAY
        schedule.every().day.at("12:00").do(send_message_to_whatsapp,name = name,mobile_number = number)
        
        # Run the scheduling check continuously
        while True:
            schedule.run_pending()
            time.sleep(1)
